In [3]:
import pandas as pd
import numpy as np
from keras import layers
from keras import models
from resnets_utils import *

In [4]:
model= models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                36928     
__________

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_X_orig = np.array(train.drop('label',axis=1))
train_y_orig = np.array(train['label'])
test_X_orig = np.array(test)
train_X = train_X_orig.reshape((42000,28,28))/255.
test_X = test_X_orig.reshape((28000,28,28))/255.
train_Y = convert_to_one_hot(train_y_orig,10).T
train_X = train_X.reshape(42000,28,28,1)

In [6]:
model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_X,train_Y,epochs=5, batch_size=64)

Epoch 1/5
42000/42000 [==============================] - 59s 1ms/step - loss: 0.2150 - acc: 0.9325
Epoch 2/5
42000/42000 [==============================] - 45s 1ms/step - loss: 0.0555 - acc: 0.9830
Epoch 3/5
42000/42000 [==============================] - 49s 1ms/step - loss: 0.0371 - acc: 0.9885
Epoch 4/5
42000/42000 [==============================] - 61s 1ms/step - loss: 0.0280 - acc: 0.9913
Epoch 5/5
42000/42000 [==============================] - 52s 1ms/step - loss: 0.0215 - acc: 0.9932


In [7]:
test_X = test_X.reshape(28000,28,28,1)
y_pred = model.predict(test_X)

In [8]:
import csv
output =[['ImageId','Label']]
predictions = np.concatenate((np.arange(1,28001).reshape(28000,1), y_pred.argmax(axis=1).reshape(28000,1)),axis=1)
for element in predictions:
    output.append(element)

with open('predictions.csv','w',newline='',encoding='utf-8') as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerows(output)